# **API-Einbindung mit Grok 3 Mini und NiceGUI**

In [ ]:
# Installiere die benötigten Bibliotheken
!pip install xai-sdk

# Installiere pyngrok für die Verbindung von NiceGUI in Colab
!pip install pyngrok

# Lade das deutsche Sprachmodell für spaCy (falls später für ABSA oder Syn-Chain benötigt)
!python -m spacy download de_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 53.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install ipywidgets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00


## Import relevanter Bibliotheken

In [2]:
from IPython.display import display, HTML
import ipywidgets as widgets
import asyncio
import nest_asyncio
from xai_sdk import Client
from xai_sdk.chat import user, system
from google.colab import userdata
import os
import json
import time
nest_asyncio.apply()


## Erstellung Client Objekts mit API Zugriff:

### API-Key in Google Colab als Umgebungsvariable setzen

In [3]:
os.environ['XAI_API_KEY'] = userdata.get('XAI_API_KEY')

print("API-Key geladen (letzte 6 Zeichen):", os.environ['XAI_API_KEY'][-4:])

API-Key geladen (letzte 6 Zeichen): NTQk


Systemprompt hinzufügen, um zu bestimmen, wie das Sprachmodell die Anfrage des Users verarbeiten soll:

In [4]:
# Erstellung Client Objekts mit API Zugriff:
client = Client(api_key=os.getenv("XAI_API_KEY")) # Annahme: API-Key wird als Umgebungsvariable gesetzt

chat = client.chat.create(model="grok-3-mini")  #Erstellung eines Chat-Objektes (global für Multi-Turn)
chat.append(system("You are Grok, an LLM and expert in aspect-based sentiment analysis for Google reviews about German nursing homes."))

messages {
  content {
    text: "You are Grok, an LLM and expert in aspect-based sentiment analysis for Google reviews about German nursing homes."
  }
  role: ROLE_SYSTEM
}
model: "grok-3-mini"

In [5]:

# Funktion für Grok Anfrage
def ask_grok(change=None):
    prompt = prompt_input.value.strip()
    if not prompt:
        output_label.value = "Please write in your prompt!"
        return
    try:
        output_label.value = "Grok is thinking..."
        chat.append(user(prompt))

        # synchrone Anfrage
        response = chat.sample()
        text = response.content if hasattr(response, 'content') else str(response)
        output_label.value = text

    except Exception as e:
        output_label.value = f"Fehler bei de API-Anfrage: {e}"

# User Interface
display(HTML("<h2 style='text-align:center; color:#1a73e8;'>Grok Chat</h2>"))

prompt_input = widgets.Textarea(
    placeholder="z.B. Whazzup?",
    description="Prompt:",
    layout=widgets.Layout(height='120px', width='100%')
)

button = widgets.Button(
    description="Ask Grok",
    button_style="primary",
    icon="paper-plane"
)
button.on_click(ask_grok)

output_label = widgets.Textarea(
    placeholder="Answer comes here...",
    description="Grok:",
    disabled=True,
    layout=widgets.Layout(height='350px', width='100%')
)

# Anzeige des UIs
display(widgets.VBox([prompt_input, button, output_label]))